In [59]:
from typing import Any, Dict, Union
import pandas as pd
import logging
import re
import sqlfluff
import requests
import ruamel.yaml
import psycopg2
from bs4 import BeautifulSoup
from markdown import markdown
import datetime
from urllib.parse import unquote
from typing import Any, Dict, Iterator, List, Union
import json
from pgsanity.pgsanity import check_string
import os
from dbt.cli.main import dbtRunner, dbtRunnerResult
import smtplib
import ssl
from itertools import compress
import numpy as np
from pathlib import Path

In [61]:
MATERIALIZATION_MAPPING = {1: "table", 2: "view", 3: "incremental", 4: "ephemereal"}
SUPERSET_USERNAME = "superset"
SUPERSET_PASSWORD = "superset"
SUPERSET_HOST = "http://34.82.185.252:30007/"
DATABASE_USERNAME = "fdp"
DATABASE_PASSWORD = "fdp"
DATABASE_HOST = "34.82.185.252"
DATABASE_PORT = 30005
DATABASE_NAME = "financial_data"
QUERY_SCHEMA="financial_query"
QUERY_TABLE="query"
MANIFEST_PATH="/home/vu/Desktop/Projects/Thesis/financial/dbt/target/manifest.json"
EMAIL_PORT = 465
SMTP = "smtp.gmail.com"
EMAIL_SENDER = "catvu113@gmail.com"
EMAIL_PASSWORD = "xhtzakhmnsbufufy"
USER_MODEL_PATH = "/home/vu/Desktop/Projects/Thesis/financial/dbt/models/user"
DBT_PROJECT_DIR = "/home/vu/Desktop/Projects/Thesis/financial/dbt/"
DATABASE_ID = 1
SUPERSET_ID = 34
USER_SCHEMA = "user"
SERVING_SCHEMA="marts"
context = ssl.create_default_context()
SMTP = smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context)
SST_DATABASE_NAME = "FDP Reader"

In [57]:
class SupersetDBTSessionConnector:
    """A class for accessing the Superset API in an easy way."""

    def __init__(self):
        """Instantiates the class.

        ''access_token'' will be instantiated via enviromental variable
        If ``access_token`` is None, attempts to obtain it using ``refresh_token``.

        Args:
            api_url: Base API URL of a Superset instance, e.g. https://my-superset/api/v1.
            access_token: Access token to use for accessing protected endpoints of the Superset
                API. Can be automatically obtained if ``refresh_token`` is not None.
            refresh_token: Refresh token to use for obtaining or refreshing the ``access_token``.
                If None, no refresh will be done.
        """
        self.__url = SUPERSET_HOST
        self.__api_url = self.__url + "api/v1/"

        self.__session = requests.session()

        self.__username = SUPERSET_USERNAME
        self.__password = SUPERSET_PASSWORD

        self.__refresh_session()

    def __refresh_session(self):
        logging.info("Refreshing session")

        soup = BeautifulSoup(self.__session.post(self.__url + "login").text, "html.parser")
        self.__csrf_token = soup.find("input", {"id": "csrf_token"})["value"]  # type: ignore

        data = {
            "username": self.__username,
            "password": self.__password,
            "provider": "db",
            "refresh": True,
        }
        headers = {
            # 'Authorization': 'Bearer {}'.format(self.____access_token),
            "x-csrftoken": self.__csrf_token,
        }
        response = self.__session.post(self.__url + "login", json=data, headers=headers)  # type: ignore
        return True

    def request(self, method, endpoint, **request_kwargs):
        """Executes a request against the Superset API.

        Args:
            method: HTTP method to use.
            endpoint: Endpoint to use.
            **request_kwargs: Any ``requests.request`` arguments to use.

        Returns:
            A dictionary containing response body parsed from JSON.

        Raises:
            HTTPError: There is an HTTP error (detected by ``requests.Response.raise_for_status``)
                even after retrying with a fresh session.
        """

        logging.info("About to %s execute request for endpoint %s", method, endpoint)

        url = self.__api_url + endpoint
        csrf_headers = {
            # 'Authorization': 'Bearer {}'.format(self.__access_token),
            "x-csrftoken": self.__csrf_token,
        }

        res = self.__session.request(method, url, headers=csrf_headers, **request_kwargs)  # type: ignore

        logging.info("Request finished with status: %d", res.status_code)

        if res.status_code == 401 and res.json().get("msg") == "Token has expired" and self.__refresh_session():
            logging.info(f"Retrying {method} request for {url} %s with refreshed session")
            res = self.__session.request(method, url, headers=csrf_headers, **request_kwargs)  # type: ignore

            logging.info("Request finished with status: %d", res.status_code)

        if (
            res.status_code == 400
            and res.json()["message"] == "400 Bad Request: The CSRF session token is missing."
            and self.__refresh_session()
        ):
            logging.info(f"Retrying {method} request for {url} %s with refreshed session")
            res = self.__session.request(method, url, headers=csrf_headers, **request_kwargs)  # type: ignore
            logging.info(f"Request finished with status: {res.status_code}")
        res.raise_for_status()
        return res.json()

In [62]:
superset = SupersetDBTSessionConnector()

if Path(MANIFEST_PATH).is_file():
    with open(MANIFEST_PATH) as f:
        dbt_manifest = json.load(f)
else:
    raise Exception("No manifest found at path")

dbt_tables = get_tables_from_dbt(dbt_manifest, None)
serving_dbt_models = [
    dbt_tables[table] for table in dbt_tables if dbt_tables[table]["schema"] in (SERVING_SCHEMA, USER_SCHEMA)
]
datasources = []
for model in serving_dbt_models:
    datasources.append(
        {
            "database_name": SST_DATABASE_NAME,
            "datasource_name": model["name"],
            "datasource_type": "table",
            "schema": model["schema"],
        }
    )
datasources = {"datasources": datasources}
superset.request("POST", "cachekey/invalidate", json=datasources)


{}

In [ ]:
force": true,
  "thumb_size": [
    0
  ],
  "window_size": [
    0
  ]

In [ ]:
superset.request("GET",f'chart/1/cache_screenshot/?q={{"page":{page_number},"page_size":100}}')

In [51]:
superset = SupersetDBTSessionConnector()
datasources = {
    "datasources": [
        {
            "database_name": "FDP Reader",
            "datasource_name": "marts.fact_price_history",
            "datasource_type": "table",
            "schema": "marts",
        }
    ]
}
superset.request("POST", "cachekey/invalidate", json=datasources)


{}

In [10]:
def is_valid_table_name(table_name):
    """
    Checks if the given string is a valid table name in PostgreSQL.

    Args:
        table_name: The string to check.

    Returns:
        True if the string is a valid table name, False otherwise.
    """

    # The regular expression to match a valid table name.
    regex = re.compile(r"^[a-zA-Z0-9_]{1,63}$")

    # Check if the string matches the regular expression.
    if regex.match(table_name):
        return True
    else:
        return False


def is_unique_table_name(table_name, dbt_tables):
    """
    Checks if the given string is a valid table name in PostgreSQL and dbt.

    Args:
        table_name: The string to check.
        dbt_tables: Dict of get_dbt_tables
    Returns:
        True if the string is a valid table name, False otherwise.
    """

    # The regular expression to match a valid table name.
    regex = re.compile(r"^[a-zA-Z0-9_]{1,63}$")

    # Check if the string matches the regular expression.
    if table_name not in dbt_tables:
        return True
    else:
        return False



def get_ref(original_query, dbt_tables, parsed_result, dbt_tables_names):
    """
    Returns content of a user-created dbt model file w/o config.

    Args:
        original_query: Query needed processing
        dbt_tables: Dict of dicts obtained by get_tables_from_dbt.
        schema_names: List of serving schema names.

    Returns:
        String: the content of the dbt model.
    """
    # original_query = original_query[:-1] if original_query[-1] == ";" else original_query # Maybe unneeded since not wrapping with
    # Access table names
    fixed_query = str(original_query)
    table_names = set(get_tables_from_sql(fixed_query, dialect="postgres", sql_parsed=parsed_result))
    fixed_query = sqlfluff.fix(fixed_query, dialect="postgres")
    if len(table_names.difference(dbt_tables_names)) > 0:  # dbt_tables_names include schema
        return None, "Tables referenced out of serving schemas"
    # Put tables in subqueries
    final_tables = tuple(table_names.intersection(dbt_tables_names))  # Filter out

    if len(final_tables) == 0:
        return None, "No tables referenced in dbt projects"

    table_to_ref = {}

    # Add ref for original query
    new_query = original_query
    for table in final_tables:
        new_query = (
            """
-- depends_on: {{{{ref(\'{table}\')}}}}
    """.format(
                table=dbt_tables[table]["name"]  # Ensure that there is only table names, no schema names
            )
            + new_query
        )
    return new_query, "Success"

def add_materialization(df_row, query, exec_time):
    """
    Returns content of a user-created dbt model file with config.

    Args:
        df_row: Row of DataFrame taken from "query" table.
        dbt_tables: List of tables name.
        schema_names: List of serving schema names.

    Returns:
        String: the content of the dbt model.
    """
    query = (
        """
{{{{ config(
    materialized=\'{materialization}\',
    name='{name}',
    description='{desc}',
    tags = ['{user_id}','user_created','{created_time}'],
    schema = 'financial_user'
) }}}}""".format(
            materialization=MATERIALIZATION_MAPPING[df_row["materialization"]],
            user_id=df_row["user_id"],
            name=df_row["name"],
            desc=df_row["description"],
            created_time=exec_time,
        )
        + query
    )
    return query


def get_records():
    # Query records
    try:
        connection = psycopg2.connect(
            user=DATABASE_USERNAME,
            password=DATABASE_PASSWORD,
            host=DATABASE_HOST,
            port=DATABASE_PORT,
            database=DATABASE_NAME,
        )
        cursor = connection.cursor()
        postgreSQL_select_Query = f"select * from {QUERY_SCHEMA}.{QUERY_TABLE} where checked = False"
        # postgreSQL_select_Query = """
        # SELECT *
        # FROM query
        # WHERE insert_time  > now() - interval '30 second';
        # """
        print(postgreSQL_select_Query)
        cursor.execute(postgreSQL_select_Query)
        query_columns = [
            "query_string",
            "materialization",
            "user_id",
            "description",
            "insert_time",
            "name",
            "checked",
            "success",
        ]

        df = pd.DataFrame(cursor.fetchall(), columns=query_columns)

    except (Exception, psycopg2.Error) as error:
        print("Error while fetching data from PostgreSQL", error)
    finally:
        # closing database connection.
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")
    return df


def update_records(update_values):
    try:
        connection = psycopg2.connect(
            user=DATABASE_USERNAME,
            password=DATABASE_PASSWORD,
            host=DATABASE_HOST,
            port=DATABASE_PORT,
            database=DATABASE_NAME,
        )
        cursor = connection.cursor()
        update_sql_query = f"""UPDATE {QUERY_SCHEMA}.{QUERY_TABLE} q 
                                SET success = v.success,
                                    checked = v.checked

                                FROM (values {update_values}) AS v (name, user_id, checked, success)
                                WHERE q.user_id = v.user_id 
                                AND q.name = v.name;"""
        print(update_sql_query)
        cursor.execute(update_sql_query)
    except (Exception, psycopg2.Error) as error:
        print("Error while updating data in PostgreSQL", error)

        cursor.execute(update_sql_query)

    finally:
        # closing database connection.
        if connection:
            cursor.close()
            connection.close()
            print("PostgreSQL connection is closed")


def get_emails(superset, user_ids):
    url = unquote(f"/security/get_email/?q={list(user_ids)}")
    res = superset.request("GET", url)
    return res["emails"]

def get_tables_from_dbt(dbt_manifest, dbt_db_name):
    
    tables = {}
    for table_type in ["nodes"]:
        manifest_subset = dbt_manifest[table_type]

        for table_key_long in manifest_subset:
            table = manifest_subset[table_key_long]
            name = table["name"]
            schema = table["schema"]
            database = table["database"]
            description = table["description"]
            alias = table["alias"]
            source = table["unique_id"].split(".")[-2]
            table_key = schema + "." + alias  # Key will be alias, not name
            columns = table["columns"]

            if dbt_db_name is None or database == dbt_db_name:
                # fail if it breaks uniqueness constraint
                assert table_key not in tables, (
                    f"Table {table_key} is a duplicate name (schema + table) across databases. "
                    "This would result in incorrect matching between Superset and dbt. "
                    "To fix this, remove duplicates or add ``dbt_db_name``."
                )
                tables[table_key] = {
                    "name": name,
                    "schema": schema,
                    "database": database,
                    "type": table_type[:-1],
                    "ref": f"ref('{name}')" if table_type == "nodes" else f"source('{source}', '{name}')",
                    "user": None,
                    "columns": columns,
                    "description": description,
                    "alias": alias,
                }
            if schema == "user":
                tables[table_key]["user"] = table["tags"][0]

    assert tables, "Manifest is empty!"

    return tables
def get_tables_from_sql_simple(sql):
    """
    (Superset) Fallback SQL parsing using regular expressions to get tables names.
    """
    sql = re.sub(r"(--.*)|(#.*)", "", sql)
    sql = re.sub(r"\s+", " ", sql).lower()
    sql = re.sub(r"(/\*(.|\n)*\*/)", "", sql)

    regex = re.compile(r"\b(from|join)\b\s+(\"?(\w+)\"?(\.))?\"?(\w+)\"?\b")
    tables_match = regex.findall(sql)
    tables = [
        table[2] + "." + table[4] if table[2] != "" else table[4] for table in tables_match if table[4] != "unnest"
    ]

    tables = list(set(tables))

    return tables


def get_tables_from_sql(sql, dialect, sql_parsed=None):
    """
    (Superset) SQL parsing using sqlfluff to get clean tables names.
    If sqlfluff parsing fails it runs the above regex parsing func.
    Returns a tables list.
    """
    try:
        if not sql_parsed:
            sql_parsed = sqlfluff.parse(sql, dialect=dialect)
        tables_raw = list(get_json_segment(sql_parsed, "table_reference"))  # type: ignore
        tables_cleaned = []  # With schema
        for table_ref in tables_raw:
            if isinstance(table_ref, list):
                table_ref_identifier = []
                # Get last 2 "naked_identifier"
                for dictionary in table_ref[::-1]:
                    if "naked_identifier" in dictionary:
                        table_ref_identifier.append(dictionary["naked_identifier"])
                        if len(table_ref_identifier) == 2:
                            tables_cleaned.append(".".join(table_ref_identifier[::-1]))
                            break
            if isinstance(table_ref, dict):
                tables_cleaned.append(table_ref["naked_identifier"])
    except (
        sqlfluff.core.errors.SQLParseError,  # type: ignore
        sqlfluff.core.errors.SQLLexError,  # type: ignore
        sqlfluff.core.errors.SQLFluffUserError,  # type: ignore
        sqlfluff.api.simple.APIParsingError,  # type: ignore
    ) as e:  # type: ignore
        logging.warning(
            "Parsing SQL through sqlfluff failed. "
            "Let me attempt this via regular expressions at least and "
            "check the problematic query and error below.\n%s",
            sql,
            exc_info=e,
        )
        tables_cleaned = get_tables_from_sql_simple(sql)

    return tables_cleaned

def get_json_segment(
    parse_result: Dict[str, Any], segment_type: str
) -> Iterator[Union[str, Dict[str, Any], List[Dict[str, Any]]]]:
    """Recursively search JSON parse result for specified segment type.

    Args:
        parse_result (Dict[str, Any]): JSON parse result from `sqlfluff.fix`.
        segment_type (str): The segment type to search for.

    Yields:
        Iterator[Union[str, Dict[str, Any], List[Dict[str, Any]]]]:
        Retrieves children of specified segment type as either a string for a raw
        segment or as JSON or an array of JSON for non-raw segments.
    """
    for k, v in parse_result.items():
        if k == segment_type:
            yield v
        elif isinstance(v, dict):
            yield from get_json_segment(v, segment_type)
        elif isinstance(v, list):
            for s in v:
                yield from get_json_segment(s, segment_type)


In [11]:
df = get_records()

select * from financial_query.query where checked = False
PostgreSQL connection is closed


In [12]:
df

,query_string,materialization,user_id,description,insert_time,name,checked,success
0,SELECT * from marts.dim_balance_sheet,2,1,,2023-07-24 09:37:06.123662,asd,False,None
1,SELECT * from dim_bollinger,2,1,,2023-07-23 08:11:58.188941,unique_2,False,None
2,SELECT * from marts.dim_bollinger,2,1,asdad,2023-07-23 08:12:43.670466,unique_3,False,None
3,SELECT * from marts.dim_balance_sheet,2,1,asdasdwa,2023-07-23 09:37:33.026336,unique_4,False,None


In [17]:
logger = logging.getLogger("create_query")
df = get_records()

dbt = dbtRunner()
cli_args = [
    "parse",
    "--project-dir",
    DBT_PROJECT_DIR,
]

# if df.empty:
#     print(df.empty)
#     return "Early stopping because no records"

# Get dagster execution time, see: https://stackoverflow.com/questions/75099470/getting-current-execution-date-in-a-task-or-asset-in-dagster
EXEC_TIME = datetime.datetime.today().strftime("%d/%m/%Y_%H:%M:%S")
# raise Exception(DBT_PROJECT_DIR, MANIFEST_PATH, USER_MODEL_PATH)
# Get all schema names in project
# Either this or defined schema name available to the user before
with open(MANIFEST_PATH) as f:
    dbt_manifest = json.load(f)
    dbt_tables = get_tables_from_dbt(dbt_manifest, None)

# Getting the dbt tables keys
dbt_tables_names = set(list(dbt_tables.keys()))
status = []  # Status of preliminary checking
dbt_names_aliases = [dbt_tables[table]["name"] for table in dbt_tables] + [
    dbt_tables[table]["alias"] for table in dbt_tables
]  # Name and aliases wo schema

for i in df.index:
    # Check name validity
    print(df.loc[i]["name"])
    name_validation = is_valid_table_name(df.loc[i]["name"])
    print(name_validation)
    if not name_validation:
        status.append("Invalid name")
        df.loc[i, "success"] = False
        continue
    name_unique = is_unique_table_name(df.loc[i]["name"], dbt_names_aliases)  # check aliases and name
    print(name_unique)
    if not name_unique:
        status.append("Model name is duplicated with another existing model")
        df.loc[i, "success"] = False
        continue
    # Check syntax
    query_string = df.loc[i]["query_string"]
    query_string = query_string + ";" if query_string[-1] != ";" else query_string
    validation = check_string(query_string)
    if not validation[0]:
        df.loc[i, "success"] = False
        status.append("Invalid query: {error}".format(error=validation[1]))
        continue
    # Check multi-query
    parsed = sqlfluff.parse(query_string, "postgres")["file"]
    if type(parsed) == list:
        df.loc[i, "success"] = False
        status.append("Multiple statement")
        continue
    # Check select statements
    # if list(statement_list[0]["statement"].keys())[0] != "select_statement":
    #     df.loc[i, "success"] = False
    #     status.append("Query is not 'SELECT'")
    #     continue
    # Check tables and add model ref
    partially_model, processed_status = get_ref(df.loc[i, "query_string"], dbt_tables, parsed, dbt_tables_names)
    if processed_status != "Success":
        df.loc[i, "success"] = False
        status.append(processed_status)
        continue
    # Add config
    processed_model = add_materialization(df.loc[i], partially_model, EXEC_TIME)

    model_path = USER_MODEL_PATH + "/{name}.sql".format(name=df.loc[i, "name"])

    with open(model_path, "w+") as f:
        f.write(processed_model)
        print("Wrote model {name} contents".format(name=df.loc[i, "name"]))
        f.close()
    status.append("Success")

# Get Emails from API
superset = SupersetDBTConnectorSession(logger=logger)
users = set(df["user_id"].to_list())
email_dict = get_emails(superset, users)

SMTP.login(EMAIL_SENDER, EMAIL_PASSWORD)

for i in df.index:
    # Check Success
    if df.loc[i, "success"] == False:
        message = """\
Subject: Superset Model Creation

Your Model was unsuccessfully created.

Reason:
{reason}

SQL:
{sql}
""".format(
            reason=status[i], sql=df.loc[i, "query_string"]
        )

        df.loc[i, "checked"] = True
        SMTP.sendmail(EMAIL_SENDER, email_dict[str(df.loc[i, "user_id"])], message)

# If every record is unsuccesful, terminate script early
# if not df["success"].any():
#     entries_to_update = str(tuple(zip(df.name, df.user_id, df.checked, df.success))).replace("None", "Null")[1:-1]
#     print("entries")
#     print(entries_to_update)
#     update_records(entries_to_update)
#     return "Early stopping because no successful records"

# initialize
dbt = dbtRunner()

# create CLI args as a list of strings
cli_args = [
    "run",
    "--project-dir",
    DBT_PROJECT_DIR,
    "--select",
    "tag:{exec_time}".format(exec_time=EXEC_TIME),
]

# run the command
res: dbtRunnerResult = dbt.invoke(cli_args)

# inspect the results
for r in res.result:
    print(f"{r.node.name}: {r.status}")
# Map df index to result
dbt_res_df_map = {}

for i in df.index:
    for r in res.result:
        if r.node.name == df.loc[i, "name"]:
            dbt_res_df_map[i] = r
            break

for i in df.index:
    # Check Success
    if df.loc[i, "success"] is not False:
        if dbt_res_df_map[i].status == "success":
            df.loc[i, "success"] = True
            rison_request = "/dataset/"
            # Data to be written
            dictionary = {
                # Parameter database
                "database": DATABASE_ID,
                "schema": USER_SCHEMA,
                "table_name": df.loc[i, "name"],
                "owners": [int(df.loc[i, "user_id"]), SUPERSET_ID],
            }
            # Serializing json
            json_object = json.dumps(dictionary)
            response = superset.request("POST", rison_request, json=dictionary)

            message = """\
Subject: Superset Model Creation

Your Model was successfully created. 

SQL:{sql}
""".format(
                sql=df.loc[i, "query_string"]
            )

        else:
            df.loc[i, "success"] = False
            message = """\
Subject: Superset Model Creation

Your Model was unsuccessfully created during dbt's run, please contact the administrator.

Reason:
{reason}

SQL:
{sql}
""".format(
                reason=dbt_res_df_map[i].message, sql=df.loc[i, "query_string"]
            )

        df.loc[i, "checked"] = True

        SMTP.sendmail(EMAIL_SENDER, email_dict[str(df.loc[i, "user_id"])], message)

# Delete unsucessful model
for i in df.index:
    # Check Success
    if not df.loc[i, "success"]:
        model_path = "models/user/{name}.sql".format(name=df.loc[i, "name"])
        if os.path.exists(model_path):
            os.remove(model_path)

entries_to_update = str(tuple(zip(df.name, df.user_id, df.checked, df.success))).replace("None", "Null")[1:-1]
print("entries")
print(entries_to_update)
update_records(entries_to_update)


select * from financial_query.query where checked = False
PostgreSQL connection is closed
asd
True
True
Wrote model asd contents
unique_2
True
True
unique_3
True
True
Wrote model unique_3 contents
unique_4
True
True
Wrote model unique_4 contents
10:44:42  Running with dbt=1.5.1
10:44:43  Found 33 models, 73 tests, 6 snapshots, 0 analyses, 760 macros, 0 operations, 0 seed files, 12 sources, 1 exposure, 0 metrics, 0 groups
10:44:43  
10:44:52  Concurrency: 4 threads (target='dev_cloud')
10:44:52  
10:44:52  1 of 3 START sql view model financial_user.asd ................................. [RUN]
10:44:52  2 of 3 START sql view model financial_user.unique_3 ............................ [RUN]
10:44:52  3 of 3 START sql view model financial_user.unique_4 ............................ [RUN]
10:44:55  2 of 3 OK created sql view model financial_user.unique_3 ....................... [CREATE VIEW in 3.00s]
10:44:55  3 of 3 OK created sql view model financial_user.unique_4 ....................... [CR

In [18]:
entries_to_update = str(tuple(zip(df.name, df.user_id, df.checked, df.success))).replace("None", "Null")[1:-1]
print("entries")
print(entries_to_update)

entries
('asd', 1, True, True), ('unique_2', 1, True, False), ('unique_3', 1, True, True), ('unique_4', 1, True, True)


In [19]:
update_records(entries_to_update)


UPDATE financial_query.query q 
                                SET success = v.success,
                                    checked = v.checked

                                FROM (values ('asd', 1, True, True), ('unique_2', 1, True, False), ('unique_3', 1, True, True), ('unique_4', 1, True, True)) AS v (name, user_id, checked, success)
                                WHERE q.user_id = v.user_id 
                                AND q.name = v.name;
PostgreSQL connection is closed


In [74]:
for i in dbt_res_df_map.keys():
    print(i)

4


In [80]:
for i in df.index:
    for r in res.result:
        print(f"{r.node.name}: {r.status}")
        print(r.node.name == df.loc[i, "name"])
# Map df index to result
dbt_res_df_map = {}

for i in df.index:
    for r in res.result:
        if r.node.name == df.loc[i, "name"]:
            dbt_res_df_map[i] = r
            break

unique_1: success
False
tetst: success
False
unique_1: success
False
tetst: success
False
unique_1: success
False
tetst: success
False
unique_1: success
False
tetst: success
True
unique_1: success
True
tetst: success
False


In [81]:
dbt_res_df_map

{3: RunResult(status=<RunStatus.Success: 'success'>, timing=[TimingInfo(name='compile', started_at=datetime.datetime(2023, 7, 23, 6, 49, 39, 256404), completed_at=datetime.datetime(2023, 7, 23, 6, 49, 39, 279021)), TimingInfo(name='execute', started_at=datetime.datetime(2023, 7, 23, 6, 49, 39, 283493), completed_at=datetime.datetime(2023, 7, 23, 6, 49, 42, 346239))], thread_id='Thread-40 (worker)', execution_time=3.102137327194214, adapter_response={'_message': 'SELECT 1979', 'code': 'SELECT', 'rows_affected': 1979}, message='SELECT 1979', failures=None, node=ModelNode(database='financial_data', schema='financial_user', name='tetst', resource_type=<NodeType.Model: 'model'>, package_name='dbt_financial', path='user/tetst.sql', original_file_path='models/user/tetst.sql', unique_id='model.dbt_financial.tetst', fqn=['dbt_financial', 'user', 'tetst'], alias='tetst', checksum=FileHash(name='sha256', checksum='8e23edf47bc6221ab2224b44712988691722cb47b9151c36229d1e70454a867a'), config=NodeConf

In [55]:
dbt_res_df_map

{3: RunResult(status=<RunStatus.Success: 'success'>, timing=[TimingInfo(name='compile', started_at=datetime.datetime(2023, 7, 23, 6, 36, 18, 918788), completed_at=datetime.datetime(2023, 7, 23, 6, 36, 18, 941524)), TimingInfo(name='execute', started_at=datetime.datetime(2023, 7, 23, 6, 36, 18, 946818), completed_at=datetime.datetime(2023, 7, 23, 6, 36, 22, 629960))], thread_id='Thread-26 (worker)', execution_time=3.7253305912017822, adapter_response={'_message': 'SELECT 1979', 'code': 'SELECT', 'rows_affected': 1979}, message='SELECT 1979', failures=None, node=ModelNode(database='financial_data', schema='financial_user', name='tetst', resource_type=<NodeType.Model: 'model'>, package_name='dbt_financial', path='user/tetst.sql', original_file_path='models/user/tetst.sql', unique_id='model.dbt_financial.tetst', fqn=['dbt_financial', 'user', 'tetst'], alias='tetst', checksum=FileHash(name='sha256', checksum='446ff4b5dcaab790e15e72b1b78d8cec877be517e2f2d157d7fcb5c8c82ed6c1'), config=NodeCon

In [61]:
dbt_res_df_map[3].status=="success"

True

In [49]:
for i in df.index:
    # Check Success
    if not df.loc[i, "success"]:
        model_path = USER_MODEL_PATH + "/{name}.sql".format(name=df.loc[i, "name"])
        if os.path.exists(model_path):
            os.remove(model_path)

In [47]:
df

,query_string,materialization,user_id,description,insert_time,name,success,checked
0,SELECT * from marts.dim_price_history,1,1,test description,2023-07-15 05:40:36.283000,test_query_hgffhgf,False,True
1,SELECT * from dim_price_history,2,1,test_desc,2023-07-15 06:40:36.283000,test_query_2_gftyf,False,True
2,SELECT * from dim_price_history,1,1,test_desc,2023-07-20 14:13:51.968099,test_query_123,False,True
3,SELECT * from marts.dim_price_history,1,1,ffgfs,2023-07-22 09:47:39.173512,tetst,False,True
4,SELECT * from marts.dim_price_history,2,1,asd,2023-07-22 09:47:39.173000,unique_1,False,True


In [35]:
res.result

RunExecutionResult(results=[RunResult(status=<RunStatus.Error: 'error'>, timing=[], thread_id='Thread-5 (worker)', execution_time=2.9518167972564697, adapter_response={}, message='Database Error in model tetst (models/user/tetst.sql)\n  syntax error at or near ")"\n  LINE 14:   );\n             ^\n  compiled Code at target/run/dbt_financial/models/user/tetst.sql', failures=None, node=ModelNode(database='financial_data', schema='financial_user', name='tetst', resource_type=<NodeType.Model: 'model'>, package_name='dbt_financial', path='user/tetst.sql', original_file_path='models/user/tetst.sql', unique_id='model.dbt_financial.tetst', fqn=['dbt_financial', 'user', 'tetst'], alias='tetst', checksum=FileHash(name='sha256', checksum='0914864f78b4327814b8b537d6a6a724dfb24533730814a7129d43b775b9f5ed'), config=NodeConfig(_extra={'name': 'tetst', 'description': 'ffgfs'}, enabled=True, alias=None, schema='financial_user', database=None, tags=['1', 'user_created', '23/07/2023_13:27:18'], meta={}, 

In [32]:
print(add_materialization(df.loc[i], "", EXEC_TIME))


{{ config(
    materialized='table',
    name='tetst',
    description='ffgfs',
    tags = ['1','user_created','23/07/2023_13:22:24'],
    schema = 'financial_user'
) }}


In [24]:
table_names = set(get_tables_from_sql("SELECT * from marts.dim_price_history", dialect="postgres"))

In [25]:
table_names.difference(dbt_tables_reporting)

set()

In [ ]:
"""
UPDATE financial_query.query q 
                                SET success = v.success,
                                    checked = v.checked

                                FROM (values ('test_query_hgffhgf', 1, True, False), ('test_query_2_gftyf', 1, True, False), ('test_query_123', 1, True, False), ('tetst', 1, True, False)) AS v (name, user_id, checked, success)
                                WHERE q.user_id = v.user_id 
                                AND q.name = v.name;
"""

In [7]:
email_dict

{'1': 'catvu113@gmail.com'}

In [10]:
email_dict[df.loc[i, "user_id"]]

KeyError: 1

In [9]:
entries_to_update = str(tuple(zip(df.name, df.user_id, df.checked, df.success))).replace("None", "Null")[1:-1]
print(entries_to_update)

('test_query_hgffhgf', 1, False, False), ('test_query_2_gftyf', 1, False, False), ('test_query_123', 1, Null, False), ('tetst', 1, Null, False)


In [7]:
update_sql_query = f"""UPDATE {QUERY_SCHEMA}.{QUERY_TABLE} q 
                        SET success = v.success,
                            checked = v.checked

                        FROM (values {entries_to_update}) AS v (name, user_id, checked, success)
                        WHERE q.user_id = v.user_id 
                        AND q.name = v.name;"""
print(update_sql_query)

UPDATE financial_query.query q 
                        SET success = v.success,
                            checked = v.checked

                        FROM (values ('test_query_hgffhgf', 1, True, False), ('test_query_2_gftyf', 1, True, False), ('test_query_123', 1, True, False), ('tetst', 1, True, False), ('unique_1', 1, True, False)) AS v (name, user_id, checked, success)
                        WHERE q.user_id = v.user_id 
                        AND q.name = v.name;


In [9]:
df

,query_string,materialization,user_id,description,insert_time,name,checked,success
0,SELECT * from dim_bollinger,2,1,,2023-07-23 08:11:58.188941,unique_2,False,None
1,SELECT * from marts.dim_bollinger,2,1,asdad,2023-07-23 08:12:43.670466,unique_3,False,None


select * from financial_query.query where checked = False
PostgreSQL connection is closed
unique_2
True
True
Wrote model unique_2 contents
unique_3
True
True
Wrote model unique_3 contents
entries
('unique_2', 1, False, Null), ('unique_3', 1, False, Null)
UPDATE financial_query.query q 
                                SET success = v.success,
                                    checked = v.checked

                                FROM (values ('unique_2', 1, False, Null), ('unique_3', 1, False, Null)) AS v (name, user_id, checked, success)
                                WHERE q.user_id = v.user_id 
                                AND q.name = v.name;
Error while updating data in PostgreSQL column "success" is of type boolean but expression is of type text
LINE 2:                                 SET success = v.success,
                                                      ^
HINT:  You will need to rewrite or cast the expression.

PostgreSQL connection is closed


InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [21]:
EXEC_TIME

'23/07/2023_15:28:01'

In [11]:
    # initialize
    dbt = dbtRunner()

    # create CLI args as a list of strings
    cli_args = [
        "run",
        "--project-dir",
        DBT_PROJECT_DIR,
        "--select",
        "tag:{exec_time}".format(exec_time=EXEC_TIME),
    ]

    # run the command
    res: dbtRunnerResult = dbt.invoke(cli_args)

08:23:04  Running with dbt=1.5.1
08:23:04  Found 34 models, 73 tests, 6 snapshots, 0 analyses, 760 macros, 0 operations, 0 seed files, 12 sources, 1 exposure, 0 metrics, 0 groups
08:23:04  
08:23:15  Concurrency: 4 threads (target='dev_cloud')
08:23:15  
08:23:15  1 of 2 START sql view model financial_user.unique_2 ............................ [RUN]
08:23:15  2 of 2 START sql view model financial_user.unique_3 ............................ [RUN]
08:23:18  1 of 2 ERROR creating sql view model financial_user.unique_2 ................... [ERROR in 3.17s]
08:23:19  2 of 2 OK created sql view model financial_user.unique_3 ....................... [CREATE VIEW in 4.27s]
08:23:21  
08:23:21  Finished running 2 view models in 0 hours 0 minutes and 16.71 seconds (16.71s).
08:23:21  
08:23:21  Completed with 1 error and 0 warnings:
08:23:21  
08:23:21  Database Error in model unique_2 (models/user/unique_2.sql)
08:23:21    relation "dim_bollinger" does not exist
08:23:21    LINE 9:     SELECT * fr

In [96]:
sql = """
SELECT * from dim_bollinger
"""
parsed=sqlfluff.parse(sql)

In [101]:
partially_model, processed_status = get_ref(sql, dbt_tables, parsed, dbt_tables_reporting)

In [102]:
partially_model

"\n    -- depends_on: {{ref('dim_bollinger')}}\n    \nSELECT * from dim_bollinger\n"

In [52]:
status

['Tables referenced out of serving schemas', 'Success']

In [70]:
rison_request="/dataset"
dictionary = {
    # Parameter database
    "database": DATABASE_ID,
    "schema": USER_SCHEMA,
    "table_name": df.loc[i, "name"],
    "owners": [int(df.loc[i, "user_id"]), SUPERSET_ID],
}
# Serializing json
json_object = json.dumps(dictionary)
response = superset.request("POST", rison_request, json=dictionary)

HTTPError: 422 Client Error: UNPROCESSABLE ENTITY for url: http://34.82.185.252:30007/api/v1/dataset/

In [73]:
response

{'id': 113,
 'result': {'database': 1,
  'owners': [1, 34],
  'schema': 'financial_user',
  'table_name': 'unique_4'}}

In [57]:
rison_request="/dataset"
dictionary = {
    # Parameter database
    "database": DATABASE_ID,
    "schema": USER_SCHEMA,
    "table_name": "unique_3",
    "owners": [int(df.loc[i, "user_id"]), SUPERSET_ID],
}
# Serializing json
json_object = json.dumps(dictionary)
response = superset.request("POST", rison_request, json=dictionary)

In [64]:
dbt = dbtRunner()
cli_args = ["parse","--project-dir",
    DBT_PROJECT_DIR,]
res = dbt.invoke(cli_args)

09:48:04  Running with dbt=1.5.1
09:48:05  Performance info: target/perf_info.json


In [31]:
df

,query_string,materialization,user_id,description,insert_time,name,checked,success
0,SELECT * from dim_bollinger,2,1,,2023-07-23 08:11:58.188941,unique_2,False,None
1,SELECT * from marts.dim_bollinger,2,1,asdad,2023-07-23 08:12:43.670466,unique_3,False,None


In [28]:
for r in res.result:
    print(f"{r.node.name}: {r.status}")

unique_2: error
unique_3: success
